In [9]:
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
from torchview import draw_graph
import matplotlib.pyplot as plt
from ffmpeg_quality_metrics import FfmpegQualityMetrics as ffqm  # for VMAF

# set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cpu


In [ ]:
# kolla mer på om detta är rimligt

model_parameters={}
model_parameters['densenet121'] = [6,12,24,16]

# growth rate
k = 32
compression_factor = 0.5

In [ ]:
"""
Ändra och ta bort:
(This code is retrieved from https://medium.com/@karuneshu21/implement-densenet-in-pytorch-46374ef91900
Author: Karunesh Upadhyay)
"""
class DenseLayer(nn.Module):
    def __init__(self, in_channels):
        """
        First 1x1 convolution generating 4*k number of channels irrespective of the total number of input channels.
        First 3x3 convolution generating k number of channels from the 4*k number of input channels.

        Args:
        in_channels (int) : # input channels to the Dense Layer, default is 3

        """
        super(DenseLayer,self).__init__()

        self.BN1 = nn.BatchNorm2d(num_features=in_channels)
        self.conv1 = nn.Conv2d( in_channels=in_channels , out_channels=4*k , kernel_size=1 , stride=1 , padding=0 , bias = False )

        self.BN2 = nn.BatchNorm2d(num_features = 4*k)
        self.conv2 = nn.Conv2d( in_channels=4*k , out_channels=k , kernel_size=3 , stride=1 , padding=1 , bias = False )

        self.relu = nn.ReLU()
        
        
    def forward(self,x):
        """
        Bottleneck DenseLayer with following operations
        (i) batchnorm -> relu -> 1x1 conv
        (ii) batchnorm -> relu -> 3x3 conv
        
        Concatenation of input and output tensor which is the main idea of DenseNet. 

        Args:
            x (tensor) : input tensor to be passed through the dense layer
        
        Attributes:
            x (tensor) : output tensor 

        """
        x_in = x
        x = self.BN1(x)
        x = self.relu(x)
        x = self.conv1(x)

        x = self.BN2(x)
        x = self.relu(x)
        x = self.conv2(x)
        
        x = torch.cat([x_in, x], 1)
        
        return x



In [ ]:
# For testing DenseNet with input size 3 x 1920 x 1080 (Full HD)

def test_DenseNet():
    x = torch.randn(1, 3, 1920, 1080)
    model = DenseNet(model_parameters['densenet121'], k, compression_factor)
    return model


model = test_DenseNet()